In [1]:
import os
import pandas as pd
from google.cloud import storage

# GCS에서 Parquet 파일 불러오기 및 형식 확인
print("\n--- 2. GCS Parquet 파일 로드 및 형식 확인 ---")
gcs_parquet_path = "gs://sprintda05_final_project/votes/accounts_blockrecord.parquet"

print(f"📂 다음 Parquet 파일을 불러옵니다: {gcs_parquet_path}")

# pandas를 사용하여 Parquet 파일 불러오기
# 'engine'을 'pyarrow'로 명시하여 확실하게 pyarrow 사용
# gcsfs가 설치되어 있다면 pandas는 gs:// 경로를 자동으로 처리
df = pd.read_parquet(gcs_parquet_path, engine='pyarrow')

print("\n✅ Parquet 파일 로드 성공!")

print("\n--- 데이터프레임 상위 5개 행 (df.head()) ---")
print(df.head())

print("\n--- 데이터프레임 정보 (df.info()) ---")
# 각 컬럼의 이름, Non-Null 개수, 데이터 타입(Dtype) 확인
df.info()

print("\n--- 데이터프레임 기술 통계 (df.describe()) ---")
# 숫자형 컬럼에 대한 통계 정보 확인
print(df.describe(include='all')) # 모든 컬럼 타입 포함

print(f"\n--- 데이터프레임 행과 열 개수 (df.shape): {df.shape} ---")

print("\n--- 데이터프레임 컬럼 목록 (df.columns) ---")
print(df.columns.tolist())


--- 2. GCS Parquet 파일 로드 및 형식 확인 ---
📂 다음 Parquet 파일을 불러옵니다: gs://sprintda05_final_project/votes/accounts_blockrecord.parquet

✅ Parquet 파일 로드 성공!

--- 데이터프레임 상위 5개 행 (df.head()) ---
   id              reason          created_at  block_user_id  user_id
0   1               그냥... 2023-05-04 23:01:53         867483   878476
1   2         친구 사이가 어색해짐 2023-05-05 01:17:08         867190   867564
2   3  나랑 관련 없는 질문을 자꾸 보냄 2023-05-05 01:50:55         875110   875261
3   6               그냥... 2023-05-05 05:21:52         883696   883511
4   7               그냥... 2023-05-05 06:40:34         871349   870177

--- 데이터프레임 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19482 entries, 0 to 19481
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             19482 non-null  int64         
 1   reason         19482 non-null  object        
 2   created_at     19482 non-null  datetime64[ns]
 3

In [2]:
import pandas as pd

def preprocess_blockrecord(df: pd.DataFrame) -> pd.DataFrame:
    """blockrecord 전처리: 자기 자신 차단 제거"""

    print("🔧 accounts_blockrecord 전처리 시작...")

    # 자기 자신 차단 식별 및 제거
    self_blocks = df['user_id'] == df['block_user_id']
    self_block_count = self_blocks.sum()

    df_clean = df[~self_blocks].copy()

    print(f"✅ 자기 자신 차단 제거: {self_block_count:,}건")
    print(f"   최종 데이터: {len(df_clean):,}건")

    return df_clean

In [4]:
import pandas as pd

def preprocess_blockrecord(df: pd.DataFrame) -> pd.DataFrame:
    """blockrecord 전처리: 자기 자신 차단 제거"""

    print("🔧 accounts_blockrecord 전처리 시작...")

    # 자기 자신 차단 식별 및 제거
    self_blocks = df['user_id'] == df['block_user_id']
    self_block_count = self_blocks.sum()

    df_clean = df[~self_blocks].copy()

    print(f"✅ 자기 자신 차단 제거: {self_block_count:,}건")
    print(f"   최종 데이터: {len(df_clean):,}건")

    return df_clean

# ✅ 테스트용 DataFrame
data = {
    'user_id': [1, 2, 3, 4],
    'block_user_id': [2, 2, 3, 5]  # user_id == block_user_id인 경우: 3만 해당
}
df = pd.DataFrame(data)

# ✅ 함수 실행
df_result = preprocess_blockrecord(df)

# ✅ 결과 확인
print('\n[전처리 결과]')
print(df_result)


🔧 accounts_blockrecord 전처리 시작...
✅ 자기 자신 차단 제거: 2건
   최종 데이터: 2건

[전처리 결과]
   user_id  block_user_id
0        1              2
3        4              5
